# 1.0 - Initial Data Exploration (EDA)

by Michael Joshua Vargas

The primary objective of this notebook is to build a predictive model for early detection. Consequently, we have defined the target variable's prediction window (e.g., 'time to event' or 'survival') to be greater than 7 days and less than 60 days. This specific timeframe is chosen to provide sufficient lead time for intervention while maintaining relevance for an 'early' detection system.

## Data Inspection and Setup

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

### Import relevant libraries

In [2]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns

import os

import sys
from pathlib import Path

### Load the dataset

In [3]:
# Get the current working directory of the notebook
notebook_dir = Path(os.getcwd())

# Navigate up one level to reach the project root directory
project_root = notebook_dir.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import from config.py
from bank_fraud.config import INTERIM_EDA_DATASET

df = pd.read_parquet(INTERIM_EDA_DATASET)

### Dataset Shape

In [4]:
print(f"""
The dataset has a total of {df.shape[0]} rows and a total of {df.shape[1]} columns.
""")


The dataset has a total of 1073778 rows and a total of 154 columns.



### Data Types
 - Check the data type of each column. Are dates stored as objects? Are numbers stored as strings?

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1073778 entries, 0 to 1073777
Columns: 154 entries, profile_id to destination_entropy_out
dtypes: datetime64[ns](11), float64(102), int64(4), object(37)
memory usage: 1.2+ GB


The `df.info()` can be slow or memory-intensive for very large datasets as it tries to compute non-null counts for all columns. Let us use alternative and more efficient ways to check data types and identify missing values with their counts:

In [6]:
# Get all the columns and their data types
pd.set_option('display.max_rows', None)

print("Column Data Types: ")
print(df.dtypes)

# Get all the columns and their data types
pd.reset_option('display.max_rows')

Column Data Types: 
profile_id                                                         object
account_no                                                         object
full_name                                                          object
username                                                           object
date_of_birth                                              datetime64[ns]
gr_card_no                                                         object
cellphone                                                          object
orig_onboarded_datetime                                    datetime64[ns]
orig_onboarded_date                                        datetime64[ns]
orig_channel                                                       object
orig_os                                                            object
origination_type                                                   object
origination_sub_type                                               object
carded_status     

Upon inspecting the data types, it's noted that specific columns (e.g., `profile_id`, `account_no`, `full_name`, `username`,`gr_card_no`, `cellphone`, `account_number`, `source_account_number`, and `destination_account_number`) are currently object type. It's important to clarify that these columns originally contained int64 numerical data. However, prior to ingestion, a cryptographic hashing function was applied to these fields for privacy preservation, converting their integer values into non-numeric, string-like representations, hence their current object data type.

### Missing Values and Their Counts/Percentages

In [7]:
# Get all the columns and their data types
pd.set_option('display.max_rows', None)

# Calculate the count of missing values for each column
missing_counts = df.isnull().sum()

# Calculate the percentage of missing values for each column
missing_percentages = (df.isnull().sum() / len(df)) * 100

# Create a DataFrame to display the results clearly
missing_info = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Percentages': missing_percentages
})

# Filter to show only columns that have missing values
missing_info = missing_info[missing_info['Missing Count'] > 0].sort_values(by='Missing Count', ascending=False)

print("\nColumns with Missing Values:")
if not missing_info.empty:
    print(missing_info)
else:
    print("No missing values found in any column.")

# Reset
pd.reset_option('display.max_rows')


Columns with Missing Values:
                                                    Missing Count  \
max_amount_PESONET_OUT                                    1073777   
min_amount_PESONET_OUT                                    1073777   
count_PESONET_OUT                                         1073777   
amount_PESONET_OUT                                        1073777   
min_amount_PESONET_IN                                     1065201   
count_PESONET_IN                                          1065201   
amount_PESONET_IN                                         1065201   
max_amount_PESONET_IN                                     1065201   
first_mob_num_date                                        1055319   
txn_velocity_accel_wk4                                    1023238   
txn_velocity_accel_wk3                                    1021701   
first_fila_date                                           1019261   
first_fila_bank_code                                      1019261   
txn_

### Check for Duplicates

Checking for duplicate records is crucial for:
   1. **Data Accuracy**: Ensures each record is unique and valid, preventing skewed analyses.
   2. **Reliable Insights**: Avoids statistical bias and prevents machine learning models from overfitting to redundant data.
   3. **Efficiency**: Reduces unnecessary storage and processing, improving performance. 
   4. **Data Integrity**: Upholds business rules by ensuring unique identifiers are truly unique.

In [8]:
f"Our dataset has {df.duplicated().sum()} duplicates."

'Our dataset has 0 duplicates.'

### Descriptive Statistics

In [9]:
# Get all the columns and their description
pd.set_option('display.max_columns', None)

df.describe()

,date_of_birth,orig_onboarded_datetime,orig_onboarded_date,date_tagged,account_status_as_off,datetime_restricted,date_restricted,first_fila_date,change_email_occurence,change_email_flag,change_mob_num_occurence,change_mob_num_flag,first_mob_num_date,txn_count_week_wk1,txn_amt_week_wk1,txn_velocity_week_wk1,txn_days_active_week_wk1,txn_count_week_wk2,txn_amt_week_wk2,txn_velocity_week_wk2,txn_days_active_week_wk2,txn_count_week_wk3,txn_amt_week_wk3,txn_velocity_week_wk3,txn_days_active_week_wk3,txn_count_week_wk4,txn_amt_week_wk4,txn_velocity_week_wk4,txn_days_active_week_wk4,txn_velocity_delta_wk2_vs_wk1,txn_velocity_delta_wk3_vs_wk2,txn_velocity_delta_wk4_vs_wk3,txn_velocity_accel_wk3,txn_velocity_accel_wk4,flag_txn_dropoff_after_wk1,txn_count_30d,txn_amt_30d,txn_velocity_30d,txn_days_active_30d,txn_count_day_volatility_30d,txn_amt_day_volatility_30d,txn_count_vol_score_wk1,txn_amt_vol_score_wk1,txn_count_vol_score_wk2,txn_amt_vol_score_wk2,txn_count_vol_score_wk3,txn_amt_vol_score_wk3,txn_count_vol_score_wk4,txn_amt_vol_score_wk4,max_txn_count_day,min_txn_count_day,max_txn_amt_day,min_txn_amt_day,avg_amt_in_day,avg_amt_out_day,avg_net_flow_amt_day,avg_inflow_outflow_ratio_day,num_same_day_cico_days,num_inflow_days,percent_inflow_same_day_out,count_INSTAPAY_IN,count_INSTAPAY_OUT,count_PESONET_IN,count_PESONET_OUT,count_total_in,count_total_out,amount_INSTAPAY_IN,amount_INSTAPAY_OUT,amount_PESONET_IN,amount_PESONET_OUT,total_amount_in,total_amount_out,max_amount_INSTAPAY_IN,max_amount_INSTAPAY_OUT,max_amount_PESONET_IN,max_amount_PESONET_OUT,min_amount_INSTAPAY_IN,min_amount_INSTAPAY_OUT,min_amount_PESONET_IN,min_amount_PESONET_OUT,txn_days_active,first_txn_date,last_txn_date,weekend_txn_count,night_txn_count,hour_entropy,weekday_entropy,min_time_btwn_txns_sec,min_time_btwn_txns_days,max_time_btwn_txns_days,avg_time_btwn_txns_days,cv_time_btwn_txns,min_txn_sessions_per_day_3min,max_txn_sessions_per_day_3min,min_txn_sessions_per_day_5min,max_txn_sessions_per_day_5min,active_days_with_txns,num_unique_source_accounts,num_unique_source_names,num_unique_destination_accounts,num_unique_destination_names,repeat_sources,total_sources,repeat_counterparty_ratio_in,repeat_destinations,total_destinations,repeat_counterparty_ratio_out,amt_from_source,rank,total_in,top_source_share_in,amt_to_destination,rank.1,total_out,top_destination_share_out,source_entropy_in,destination_entropy_out
count,1073778,1073778,1073778,96962,1073778,69404,69404,54517,1.073778e+06,1.073778e+06,1.073778e+06,1.073778e+06,18459,300223.000000,3.002230e+05,300223.000000,300223.000000,153407.000000,1.534070e+05,153407.000000,153407.000000,142408.000000,1.424080e+05,142408.000000,142408.000000,156258.000000,1.562580e+05,156258.000000,156258.000000,94784.000000,74433.000000,79567.000000,52077.000000,50540.000000,428228.000000,428228.000000,4.282280e+05,428228.000000,428228.000000,231881.000000,2.318810e+05,88772.000000,88772.000000,58863.000000,58863.000000,56626.000000,56626.000000,69639.000000,69639.000000,428228.000000,428228.000000,4.282280e+05,4.282280e+05,4.282280e+05,428228.000000,4.282280e+05,4.282280e+05,428228.000000,428228.000000,428228.000000,353015.000000,248716.000000,8577.000000,1.0,428228.000000,428228.000000,3.530150e+05,2.487160e+05,8.577000e+03,1.0,4.282280e+05,4.282280e+05,353015.000000,248716.000000,8.577000e+03,1.0,353015.000000,248716.000000,8.577000e+03,1.0,428228.000000,428228,428228,211417.000000,261540.000000,428228.000000,428228.000000,2.543290e+05,254329.000000,254329.000000,254329.000000,176590.000000,428228.000000,428228.00000,428228.000000,428228.000000,428228.000000,356440.000000,356440.000000,248716.000000,248716.000000,143863.000000,143863.000000,143863.000000,73945.000000,73945.00000,73945.000000,3.564400e+05,356440.0,3.564400e+05,356440.000000,2.487160e+05,248716.0,2.487160e+05,248716.000000,356440.000000,248716.000000
mean,1990-03-30 14:02:27.610400256,2025-03-05 12:29:35.365884672,2025-03-04 21:49:00.480434688,2025-03-23 04:22:48.

### Data Dictionary Creation

Each entry in this dictionary provides a crucial piece of the puzzle, detailing not just the technical specifications of a feature, but also its business context and relevance to identifying fraudulent activities.

In [14]:
# --- 1. Adjust python Path to import from bank_fraud/ ---
# add 'bank_fraud' directory to the Python path
bank_fraud_path = project_root / 'bank_fraud'

# Add bank_fraud to sys.path if it's not already there
if str(bank_fraud_path) not in sys.path:
    sys.path.insert(0, str(bank_fraud_path)) # Insert at the beginning for priority

# --- 2. Import the function ---
# Import from bank_fraud.utils.data_dictionary_generator
from bank_fraud.utils.data_dictionary_generator import run_data_dictionary_pipeline

exported_path = run_data_dictionary_pipeline(df, output_dir_name="references")
print(f"\nData dictionaries generated and exported to: {exported_path.relative_to(project_root)}")


Data dictionaries generated and exported to: references


**What You'll Find Here:**

Each entry in this dictionary provides a crucial piece of the puzzle, detailing not just the technical specifications of a feature, but also its business context and relevance to identifying fraudulent activities.

*   **`feature_name`**: The unique identifier for each data point.
*   **`data_type`**: The technical format of the data (e.g., `int64`, `float64`, `object`, `datetime64[ns]`), crucial for proper data handling and analysis.
*   **`description`**: A concise explanation of what the feature represents in a business context, highlighting its potential role in fraud detection. This is where the story of our data truly begins.
*   **`null_count`**: The number of missing values, indicating data completeness and potential areas for imputation or cleaning.
*   **`unique_count`**: The number of distinct values, offering insights into the variability and cardinality of the feature.

Beyond these core attributes, we provide additional, type-specific metadata to enrich our understanding:


For numerical features (e.g., transaction amounts, frequencies, durations), we delve into their statistical distribution. These metrics help us understand typical behaviors, identify outliers, and quantify the scale of various activities.

*   **`count`**: The number of non-null observations.
*   **`mean`**: The average value, indicating central tendency.
*   **`std`**: The standard deviation, measuring the dispersion or spread of data.
*   **`min` / `max`**: The minimum and maximum values, defining the range of the data.
*   **`25%` / `50%` (Median) / `75%`**: Quartiles that provide insights into the distribution and potential skewness of the data.

#### Numeric Features: Quantifying Behavior

In [26]:
from bank_fraud.config import DATA_DICTIONARIES_DIR

pd.set_option('display.max_rows', None)

# bank-fraud-detection-project\references\numeric_data_dictionary.csv
num_path = DATA_DICTIONARIES_DIR / 'numeric_data_dictionary.csv'
num_df = pd.read_csv(num_path)

num_df

,feature_name,data_type,description,null_count,unique_count,count,mean,std,min,25%,50%,75%,max
0,change_email_occurence,int64,Contact information change frequency indicatin...,0,11,1073778.0,0.012293,0.135366,0.000000,0.000000,0.000000,0.000000,2.100000e+01
1,change_mob_num_occurence,int64,Contact information change frequency indicatin...,0,13,1073778.0,0.019025,0.181945,0.000000,0.000000,0.000000,0.000000,9.300000e+01
2,txn_count_week_wk1,float64,Weekly transaction count for wk1 showing accou...,773555,468,300223.0,3.451411,19.134752,1.000000,1.000000,1.000000,2.000000,7.060000e+02
3,txn_amt_week_wk1,float64,Weekly transaction amount for wk1 indicating s...,773555,30872,300223.0,11721.815632,70351.216397,0.010000,101.000000,602.000000,4540.000000,1.605500e+07
4,txn_velocity_week_wk1,float64,Transaction velocity (transactions per day) fo...,773555,468,300223.0,0.493059,2.733536,0.142857,0.142857,0.142857,0.285714,1.008571e+02
5,txn_days_active_week_wk1,float64,Number of active transaction days in wk1 showi...,773555,7,300223.0,1.507756,0.992050,1.000000,1.000000,1.000000,2.000000,7.000000e+00
6,txn_count_week_wk2,float64,Weekly transaction count for wk2 showing accou...,920371,528,153407.0,5.748440,30.902635,1.000000,1.000000,1.000000,3.000000,9.360000e+02
7,txn_amt_week_wk2,float64,Weekly transaction amount for wk2 indicating s...,920371,27490,153407.0,19034.017170,101769.678759,0.010000,485.000000,2000.000000,7700.000000,1.924500e+07
8,txn_velocity_week_wk2,float64,Transaction velocity (transactions per day) fo...,920371,528,153407.0,0.821206,4.414662,0.142857,0.142857,0.142857,0.428571,1.337143e+02
9,txn_days_active_week_wk2,float64,Number of active transaction days in wk2 showi...,920371,7,153407.0,1.733500,1.236126,1.000000,1.000000,1.000000,2.000000,7.000000e+00


#### Categorical Features: Classifying Patterns

Categorical features (e.g., transaction types, channels, flags) help us segment our data and uncover distinct patterns. For these, we focus on understanding their diversity and common occurrences.

*   **`other_categories_count`**: The count of less frequent categories, indicating the long tail of unique values.
*   **`min_value` / `max_value`**: For numerical-like categorical data (e.g., encoded categories), these show the range.
*   **`date_range_days`**: For date-time related categorical features, the span in days.
*   **`value_range`**: For numerical-like categorical data, the difference between max and min.

In [27]:
# bank-fraud-detection-project\references\categorical_data_dictionary.csv
categorical_path = DATA_DICTIONARIES_DIR / 'categorical_data_dictionary.csv'
categorical_df = pd.read_csv(categorical_path)

categorical_df

,feature_name,data_type,description,null_count,unique_count,min_value,max_value,date_range_days,other_categories_count,value_range
0,date_of_birth,datetime64[ns],Customer birth date for age-based risk profili...,0,22796,1925-01-25 00:00:00,2007-05-12 00:00:00,30057.0,NaN,NaN
1,orig_onboarded_datetime,datetime64[ns],Account opening timestamp for tenure analysis ...,0,1071543,2025-01-01 00:07:34.340000,2025-05-12 23:59:49.056000,131.0,NaN,NaN
2,orig_onboarded_date,datetime64[ns],Account opening timestamp for tenure analysis ...,0,132,2025-01-01 00:00:00,2025-05-12 00:00:00,131.0,NaN,NaN
3,orig_channel,object,Account opening channel (online/branch) indica...,0,3,NaN,NaN,NaN,0.0,NaN
4,orig_os,object,Operating system used during account opening i...,0,3,NaN,NaN,NaN,0.0,NaN
5,origination_type,object,Type of account origination process indicating...,0,3,NaN,NaN,NaN,0.0,NaN
6,origination_sub_type,object,Categorical information for origination sub ty...,0,6,NaN,NaN,NaN,0.0,NaN
7,carded_status,object,Debit/credit card information indicating payme...,0,2,NaN,NaN,NaN,0.0,NaN
8,card_type,object,Debit/credit card information indicating payme...,0,2,NaN,NaN,NaN,0.0,NaN
9,orig_primary_source_of_funds,object,Declared primary income source for AML complia...,0,10,NaN,NaN,NaN,0.0,NaN


#### Identifier Features: Tracking Entities

Identifier features (e.g., account numbers, profile IDs) are critical for linking information across the dataset and ensuring data integrity. They are the backbone for tracking individual entities and their activities.

*   **`total_count`**: The total number of records in the dataset.
*   **`uniqueness_ratio`**: The proportion of unique values, indicating how well a feature serves as an identifier.
*   **`duplicate_count`**: The number of duplicate entries, highlighting potential data quality issues or expected repetitions.
*   **`is_primary_key`**: A boolean flag indicating if the feature can serve as a unique identifier for each record (no duplicates, no nulls).


In [29]:
# bank-fraud-detection-project\references\identifier_data_dictionary.csv
identifier_path = DATA_DICTIONARIES_DIR / 'identifier_data_dictionary.csv'
identifier_df = pd.read_csv(identifier_path)

identifier_df

,feature_name,data_type,description,null_count,unique_count,total_count,uniqueness_ratio,duplicate_count,is_primary_key
0,profile_id,object,Unique customer identifier for linking account...,0,1073778,1073778,1.000000,0,True
1,account_no,object,Primary account number for transaction trackin...,0,1073778,1073778,1.000000,0,True
2,full_name,object,Customer full name for identity verification a...,0,1073390,1073778,0.999639,388,False
3,username,object,Digital platform username indicating online ba...,0,1073778,1073778,1.000000,0,True
4,gr_card_no,object,Debit/credit card information indicating payme...,0,1073162,1073778,0.999426,616,False
5,cellphone,object,Registered mobile number for customer contact ...,0,1073778,1073778,1.000000,0,True
6,account_number,object,Categorical information for account number sup...,0,428229,1073778,0.398806,645549,False
7,source_account_number,object,Categorical information for source account num...,0,264096,1073778,0.245950,809682,False
8,destination_account_number,object,Categorical information for destination accoun...,0,221439,1073778,0.206224,852339,False


**Identifier Data Types: Hashing for Privacy**

Some identifier columns, like `profile_id` or `account_no`, `gr_card_no`, `cellphone`, `account_number`, `source_account_number`, and `destination_account_number` appear as `object` (string) types instead of integers. This is because they have been **hashed** to protect sensitive customer identities. Hashing anonymizes the data while still allowing these columns to serve as unique identifiers for analytical purposes.

### 